<a href="https://colab.research.google.com/github/ChristianJJohnson/DS-Unit-1-Sprint-3-Linear-Algebra/blob/master/DS-Unit-1-Build-1/Christian_Johnson_DSPT4_U1_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rays of Hope: What is the true potential of solar energy?

*Don't Let the Perfect Be the Enemy of the Good*

Goals:<br> Look at the energy potential of solar energy,<br>
Visualize that potential,<br>
dive deeper into solar data to support that potential through visualizations

## Setup Code

Recently more than 11,000 researchers from around the world issued a grim warning of the "untold suffering" that will be caused by climate change if humanity doesn't change its ways. That message repeated in my head working to help me think of any ideas that could help us lessen the burdens of humans negative effects on climate change. While pondering the topic on a usually warm (frankly these days it’s becoming more expected) Louisiana Christmas the idea shined on me. That idea was the Sun, I started asking myself is solar energy a viable option that we could actually use to transform our fossil fuel dependent energy grid to.

In [0]:
# Standard non-visual imports
import numpy as np
import pandas as pd
import datetime

# plotly graphs
import plotly as py
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None  # Get rid of SettingWithCopyWarning warning on my graphs

In [0]:
# upload visualizations to plotly account
import chart_studio
import chart_studio.plotly as chart

username = 'Christian.Johnson1'
api_key = 'dLdWw3RNOxwrg3g1bVPO' 
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

### Current Adoption

In [0]:
# Data to see where current solar energy plants are
curr_df = pd.read_excel('https://query.data.world/s/qukue5lislglpflhvwaxmz4yq6jgmc', 'Operating',skiprows=1)

In [0]:
curr_df.head()

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),Net Winter Capacity (MW),Technology,Energy Source Code,Prime Mover Code,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,Status,Planned Derate Year,Planned Derate Month,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
0,195,Alabama Power Co,2.0,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,53,53,Conventional Hydroelectric,WAT,HY,7.0,1963.0,,,(OP) Operating,,,,,,,Tuscaloosa,33.458665,-87.35682,Map,Map,SOCO
1,195,Alabama Power Co,3.0,Barry,Electric Utility,AL,1,NaN,153.1,55.5,55.5,Natural Gas Steam Turbine,NG,ST,2.0,1954.0,,,(OP) Operating,,,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
2,195,Alabama Power Co,3.0,Barry,Electric Utility,AL,2,NaN,153.1,55.5,55.5,Natural Gas Steam Turbine,NG,ST,7.0,1954.0,,,(OP) Operating,,,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
3,195,Alabama Power Co,3.0,Barry,Electric Utility,AL,4,NaN,403.7,362,362,Conventional Steam Coal,BIT,ST,12.0,1969.0,,,(OP) Operating,,,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
4,195,Alabama Power Co,3.0,Barry,Electric Utility,AL,5,NaN,788.8,756.5,756.5,Conventional Steam Coal,BIT,ST,10.0,1971.0,,,(OP) Operating,,,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO


In [0]:
curr_df['Technology'].unique()

array(['Conventional Hydroelectric', 'Natural Gas Steam Turbine',
       'Conventional Steam Coal', 'Natural Gas Fired Combined Cycle',
       'Natural Gas Fired Combustion Turbine', 'Petroleum Liquids',
       'Natural Gas Internal Combustion Engine', 'Nuclear',
       'Onshore Wind Turbine', 'Hydroelectric Pumped Storage',
       'Solar Photovoltaic', 'Geothermal', 'Landfill Gas', 'Batteries',
       'Wood/Wood Waste Biomass',
       'Coal Integrated Gasification Combined Cycle', 'Other Gases',
       'Petroleum Coke', 'Municipal Solid Waste',
       'Natural Gas with Compressed Air Storage', 'All Other',
       'Other Waste Biomass', 'Solar Thermal without Energy Storage',
       'Other Natural Gas', 'Solar Thermal with Energy Storage',
       'Flywheels', 'Offshore Wind Turbine', nan], dtype=object)

In [0]:
# Filter out all technologys except solar
curr = curr_df[curr_df['Technology'].isin(['Solar Photovoltaic','Solar Thermal without Energy Storage','Solar Thermal with Energy Storage'])]

# Use later in plotly bar graph that was giving me some trouble not representing correct value counts
curr_pv = curr_df[curr_df['Technology'].isin(['Solar Photovoltaic'])]
curr_cspn = curr_df[curr_df['Technology'].isin(['Solar Thermal without Energy Storage'])]
curr_csp = curr_df[curr_df['Technology'].isin(['Solar Thermal with Energy Storage'])]

In [0]:
curr.describe(include='all')

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),Net Winter Capacity (MW),Technology,Energy Source Code,Prime Mover Code,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,Status,Planned Derate Year,Planned Derate Month,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
count,3688.0,3688,3688.000000,3688,3688,3688,3688,1,3688.000000,3688.0,3688.0,3688,3688,3688,3688.000000,3688.000000,3688,3688,3688,3688,3688,3688,3688,3688,3688,3688,3688.000000,3688.000000,3688,3688,3653
unique,1450.0,1450,NaN,3057,7,47,2263,1,NaN,308.0,309.0,3,1,3,NaN,NaN,6,5,2,1,1,1,3,3,2,570,NaN,NaN,1,1,47
top,61060.0,Cypress Creek Renewables,NaN,Solar Photovoltaic Project #26,IPP Non-CHP,CA,1,WT,NaN,1.0,1.0,Solar Photovoltaic,SUN,PV,NaN,NaN,,,(OP) Operating,,,,,,,San Bernardino,NaN,NaN,Map,Map,CISO
freq,182.0,182,NaN,12,3080,801,471,1,NaN,463.0,451.0,3669,3688,3669,NaN,NaN,3675,3675,3687,3688,3688,3688,3686,3686,3686,185,NaN,NaN,3688,3688,726
mean,NaN,NaN,59610.885846,NaN,NaN,NaN,NaN,NaN,9.493845,NaN,NaN,NaN,NaN,NaN,7.827007,2015.076193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.990552,-93.523510,NaN,NaN,NaN
std,NaN,NaN,4786.993151,NaN,NaN,NaN,NaN,NaN,22.126366,NaN,NaN,NaN,NaN,NaN,3.791974,2.989521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.563019,19.249050,NaN,NaN,NaN
min,NaN,NaN,141.000000,NaN,NaN,NaN,NaN,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,1.000000,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.766700,-159.581700,NaN,NaN,NaN
25%,NaN,NaN,58614.250000,NaN,NaN,NaN,NaN,NaN,1.100000,NaN,NaN,NaN,NaN,NaN,5.000000,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.766042,-114.962800,NaN,NaN,NaN
50%,NaN,NaN,60073.500000,NaN,NaN,NaN,NaN,NaN,2.200000,NaN,NaN,NaN,NaN,NaN,9.000000,2016.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.062856,-87.234750,NaN,NaN,NaN
75%,NaN,NaN,61379.000000,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,12.000000,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.938221,-76.914955,NaN,NaN,NaN


In [0]:
curr.info

<bound method DataFrame.info of       Entity ID  ... Balancing Authority Code
286       16572  ...                      SRP
1115      18454  ...                      TEC
1571       7095  ...                     MISO
1646      16179  ...                      PJM
2002      14201  ...                     MISO
...         ...  ...                      ...
22432     62998  ...                      PJM
22433     62998  ...                      PJM
22434     62994  ...                      PJM
22435     62994  ...                     ISNE
22440     62999  ...                      PJM

[3688 rows x 31 columns]>

In [0]:
# Going to make a visualization on state current Nameplate Capacity
states = curr['Plant State'].unique().astype(str)
key = []
val = []
for state in states:
    hold = curr[curr['Plant State'] == state]
    key.append(state)
    val.append(hold['Nameplate Capacity (MW)'].sum())

state_cap = pd.DataFrame({
    'State': key,
    'Nameplate Capacity (MW)': val
})
state_cap.head()

,State,Nameplate Capacity (MW)
0,AZ,2108.2
1,FL,1975.5
2,IL,40.9
3,IA,13.8
4,KY,26.3


### Potential

In [0]:
# checking out the data first
# Don't need it to run always
# !curl 'https://query.data.world/s/pzqmdwdvfqktvvr6fyzaahx7wnq37k'
SOL_POTEN = 'https://query.data.world/s/pzqmdwdvfqktvvr6fyzaahx7wnq37k'

In [0]:
columns = ['State', 'UrbanPVDeployment_GWh', 'UrbanPVDeployment_GW',
       'UrbanPVDeployment_km2', 'RuralPVDeployment_GWh', 'RuralPVDeployment_GW',
       'RuralPVDeployment_km2', 'RooftopPV_GWh', 'RooftopPV_GW', 'CSP_GWh', 'CSP_GW', 'CSP_km2']
# Only use the columns that pretain to solar
poten = pd.read_csv(SOL_POTEN, header=None, names=columns, skiprows=1, usecols=columns)

In [0]:
poten.insert(1, 'Code', ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"])

In [0]:
poten.head()

,State,Code,UrbanPVDeployment_GWh,UrbanPVDeployment_GW,UrbanPVDeployment_km2,RuralPVDeployment_GWh,RuralPVDeployment_GW,RuralPVDeployment_km2,RooftopPV_GWh,RooftopPV_GW,CSP_GWh,CSP_GW,CSP_km2
0,Alabama,AL,35850,20,426,3706838,2114,44058,15475.0,12,0,0,0
1,Alaska,AK,166,0,2,8282976,9005,187608,NaN,1,0,0,0
2,Arizona,AZ,121305,52,1096,11867693,5147,107230,22736.0,14,12544333,3527,107238
3,Arkansas,AR,28960,15,332,4986388,2747,57239,8484.0,6,0,0,0
4,California,CA,246008,111,2320,8855917,4010,83549,106411.0,75,8490916,2725,82859


In [0]:
# poten.isnull().sum() nulls are what's expected for the data

## Visualizations

### Current Adoption of Solar

In order to fully understand where we could go we have to know where we started.  According to the U.S. Energy Information Administration (EIA), about 4,171 billion kilowatt hours of electricity were generated at utility-scale electricity generation facilities in the United States. Only 703 billion kilowatts of that energy was generated by renewable energy sources. An even smaller amount of that was generated by Solar (64 billion kilowatts hours) around the United States.

I then wanted to see just where the facilities were spread across the country. The EIA has an annual Electric Generator Report that collects generator-level specific information about existing at electric power plants with 1 megawatt or greater of combined nameplate capacity.

What I found is that there are 3,688 solar facilities operating in 46 states around the country. California leads the charge with a whopping 801 solar facitites. Currently these facilities generate about 3.5 million kilowatt in Nameplate capacity with a Net Capacity of about 3.46 million kilowatt and 3.39 million kilowatts respectively in the Summer and Winter.


In [0]:
# Graph thoses facilities

# Display on the hover
curr['Text'] = curr['Plant Name'] + '<br>Technology: ' + (curr['Technology']).astype(str) +'<br>Nameplate Capacity: ' + (curr['Nameplate Capacity (MW)']).astype(str) +' MW'  + '<br>Net Summer Capacity: '+ (curr['Net Summer Capacity (MW)']).astype(str)+ ' MW' + '<br>Net Winter Capacity ' + (curr['Net Winter Capacity (MW)']).astype(str) +' MW'

fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = curr['Longitude'],
        lat = curr['Latitude'],
        text = curr['Text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'thermal',
            cmin = 0,
            color = curr['Nameplate Capacity (MW)'],
            cmax = curr['Nameplate Capacity (MW)'].max(),
            colorbar_title="Nameplate Capacity (MW)"
        )))

fig.update_layout(
        title = 'Current Operational Solar Energy Facilities',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
# chart.plot(fig, filename = 'current_solar_fac', auto_open=True) too big
fig.show()

In [0]:
# Data to show what the Unite states power comsumption is now
# going to come back if I have time consum = pd.read_csv('https://query.data.world/s/yu3anv66b44tgkxt3ka4w74jfzcpox')

In [0]:
# Plotly wasn't reading value counts correctly made dfs  to work from instead
# Solar Photovoltaic
df_pv = pd.DataFrame({'Plants': curr_pv['Plant State'].value_counts()})
df_pv['State'] = df_pv.index
# Solar Thermal without Energy Storage
df_cspn = pd.DataFrame({'Plants': curr_cspn['Plant State'].value_counts()})
df_cspn['State'] = df_cspn.index
# Solar Thermal with Energy Storage
df_csp = pd.DataFrame({'Plants': curr_csp['Plant State'].value_counts()})
df_csp['State'] = df_csp.index


fig = go.Figure(data=[
    go.Bar(name='Solar Photovoltaic', x=df_pv['State'], y=df_pv['Plants']),
    go.Bar(name='Solar Thermal w/ Energy Storage', x=df_csp['State'], y=df_csp['Plants']),
    go.Bar(name='Solar Thermal w/o Energy Storage', x=df_cspn['State'], y=df_cspn['Plants']),
])
# Change the bar mode
fig.update_layout(
    barmode = 'stack',
    title_text = 'Operating Solar Utility-Scale Facilities in the United States',
    xaxis = dict(
        title='States',
        titlefont_size=16,
        tickfont_size=14,
    ),

    yaxis = dict(
        title='Facilities',
        titlefont_size=16,
        tickfont_size=14,
    ),
)
# chart.plot(fig, filename = 'oper_sol_fac', auto_open=True)
fig.show()

In [0]:
fig = go.Figure(data=go.Choropleth(
    locations=state_cap['State'],
    z=state_cap['Nameplate Capacity (MW)'],
    locationmode='USA-states',
    colorscale='turbid',
    autocolorscale=False,
     # hover text text=poten['text'],
    marker_line_color='white', # line markers between states
    colorbar_title="Nameplate Capacity (MW)"
))

fig.update_layout(
    title_text='Total Nameplate Capacity By State of Current Operating Solar Facilities',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
# chart.plot(fig, filename = 'state_nameplate', auto_open=True)
fig.show()

### Potential of Solar

Naturally we have a long why to go when it comes to utilitiy scale generation at Solar facilities to meet the ever growing needs of the United States let alone the world. Next I wanted to find out just where we could go and what that potential looked like. For that I started looking at a study done by The National Renewable Energy Laboratory (NREL) a national laboratory of the U.S. Department of Energy, the study estimates the technical
potential of solar(among others) electricity generation technologies. Taking into account the potential of these technologies, they are measured on the availability, quality and technical performance of solar technologies. Also taken into account is the topographic, environmental, and land-use constraints. The study makes no indications of economic or market limitation.

In [0]:
# Change my values to Megawatt hours so it's more consistency throughout the graphs
poten['UrbanPVDeployment_MWh'] = poten['UrbanPVDeployment_GWh'].apply(lambda x: x*1000)
poten['RuralPVDeployment_MWh'] = poten['RuralPVDeployment_GWh'].apply(lambda x: x*1000)
poten['RooftopPV_MWh'] = poten['RooftopPV_GWh'].apply(lambda x: x*1000)
poten['CSP_MWh'] = poten['CSP_GWh'].apply(lambda x: x*1000)
# poten.head()
poten['UrbanPVDeployment_KWh'] = poten['UrbanPVDeployment_MWh'].apply(lambda x: x*1000)
poten['RuralPVDeployment_KWh'] = poten['RuralPVDeployment_MWh'].apply(lambda x: x*1000)
poten['RooftopPV_KWh'] = poten['RooftopPV_MWh'].apply(lambda x: x*1000)
poten['CSP_KWh'] = poten['CSP_MWh'].apply(lambda x: x*1000)

In [0]:
poten['RuralPVDeployment_KWh'].sum()

280613190000000

First the study looked at the potential of utility-scale for solar photovoltaic. You may wonder what exactly utility-scale means for solar. The general gist of it is a facility that generates power centrally and feeds it into the grid, supplying some kind of utility with energy. The visualizations show the potential for urban and rural deployment around the country. The study estimates the total annual technical potential to be 2,232 terawatt-hours for urban development while rural development has a potential of 280,613 terawatt-hours.

In [0]:
# Urban Utility Scale Deployment for PV
fig = go.Figure(data=go.Choropleth(
    locations=poten['Code'],
    z=poten['UrbanPVDeployment_KWh'],
    locationmode='USA-states',
    colorscale='thermal',
    autocolorscale=False,
     # hover text text=poten['text'],
    marker_line_color='white', # line markers between states
    colorbar_title="Kilowatts Per Hour"
))

fig.update_layout(
    title_text='Estimated Potential For Urban Deployment Of Photovoltaics',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
# chart.plot(fig, filename = 'poten_urban_pv', auto_open=True)
fig.show()


In [0]:
# Rural Utility Scale Deployment for PV
fig = go.Figure(data=go.Choropleth(
    locations=poten['Code'],
    z=poten['RuralPVDeployment_KWh'],
    locationmode='USA-states',
    colorscale='thermal',
    autocolorscale=False,
     # hover text text=poten['text'],
    marker_line_color='white', # line markers between states
    colorbar_title="Kilowatts Per Hour"
))

fig.update_layout(
    title_text='Estimated Potential For Rural Deployment Of Photovoltaics',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
# chart.plot(fig, filename = 'poten_rural_pv', auto_open=True)
fig.show()


Next up they looked at rooftop photovoltaic, the estimates were made by using a combination of floor space estimates for commercial and residential buildings and scaling by building footprints based on the average number of floors. The total annual technical potential for rooftop PV is estimated at 818 terawatt-hours. States with the largest technical potential typically have the largest populations and good solar resources. 

In [0]:
# Utility Scale Deployment of PV on Rooftops
fig = go.Figure(data=go.Choropleth(
    locations=poten['Code'],
    z=poten['RooftopPV_KWh'],
    locationmode='USA-states',
    colorscale='thermal',
    autocolorscale=False,
     # hover text text=poten['text'],
    marker_line_color='white', # line markers between states
    colorbar_title="Kilowatts Per Hour"
))

fig.update_layout(
    title_text='Estimated Potential For Rooftop Photovoltaics',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
# chart.plot(fig, filename = 'poten_rooftop_pv', auto_open=True)
fig.show()


Last but defintily not least we look at concentrating solar power sometimes known as solar thermal. It's a process where the heat of the sun is used by consentrating that heat on a lenses or mirrors and using a sun tracking systems to generate electricity to steam-driven turbines.  There is a steep cutoff of
potential, which can be attributed to the resource minimum threshold of
5 kWh/m2/day that was used in the analysis. Still the total estimated U.S. annual technical potential for CSP is 116,146 terawatt-hours.

In [0]:
#  Power from a utility-scale solar power facility
fig = go.Figure(data=go.Choropleth(
    locations=poten['Code'],
    z=poten['CSP_KWh'],
    locationmode='USA-states',
    colorscale='thermal',
    autocolorscale=False,
     # hover text text=poten['text'],
    marker_line_color='white', # line markers between states
    colorbar_title="Kilowatts Per Hour"
))

fig.update_layout(
    title_text='Estimated Potential For Concentrating Solar Power',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
# chart.plot(fig, filename = 'poten_csp', auto_open=True)
fig.show()


## Conclusion

The data shows that the potential that solar technologies provides is abundant. What may suprise you is that Texas has the most total potential in most of the technologies (all except Rooftop PV) but currently only has 57 current operational solar facilities that may suggest that there may be other factors in play such as political or economic constraints. In all, the potential to meet the current and future United States power needs through solar is ever present. The biggest hurdles we may be facing with the adoption of the technology is the initial infurstruture and status quo. But if we want to heed the warnings laid out by the world's researchers drastic changes must be made.